# Исследование рынка труда для начинающих аналитиков данных

Поиск работы для начинающих специалистов в сфере IT является одной из самых актуальных тем на рынке труда - помимо общей популярности этой индустрии для первичного трудоустройства, она также является основным направлением для переквалификации уже состоявшихся специалистов из других сфер. Такое большой спрос на вакансии сильно усложняет процесс отбора для нанимающей компании - только на первичный скрининг потенциальных кандидатов может тратиться большая часть рабочего времени специалистов по подбору персонала. Одно из популярных решений последнего времени - размещение вакансий в тематических каналах по поиску работу в мессенджере [Telegram](https://telegram.org/) - это позволяет не только проводить таргетированный поиск в профильных сообществах и размещать вакансии в специализированных карьерных каналах, но и снизить затраты на отбор за счёт оптимизации использования популярных и дорогостоящих агрегаторов вакансий и резюме.

**Цель исследования** - собрать данные о вакансиях специалистов в области науки о данных, проанализировать требования к соискателям и составить портрет идеального кандидата для трудоустройства на должность младшего аналитика, а также определить актуальность Telegram как оптимального инструмента для поиска работы.

**Ход исследования**

Данные будут получены с помощью парсинга данных карьерных каналов Telegram для кандидатов на должности *Data/Product/BI Analyst*. В ходе обзора и предобработки данных будут совершены проверки на дубликаты, пропущенные значения, корректность типов данных, после чего будет оценена необходимость в дополнительном сборе данных из внешних источников. На основе обработанных данных будет проведён исследовательский анализ, в ходе которого будут изучены статистические распределение параметров и будет установлен актуальный период исследования, после чего данные будут проанализированы только в разрезе младших специалистов. На основе выводов предыдущих этапов будет составлен портрет идеального кандидата для должности младшего специалиста и проверены следующие гипотезы:
- *Большинство работадателей готовы рассматривать кандидатов без опыта работы*
- *Большинство работадателей ждут от кандидата профильное образование* 

Результаты исследования будут представлены в виде интерактивного дашборда и презентации.

Таким образом, исследование будет разбито на следующие этапы:

1. [**Сбор и предобработка данных**](#1)
2. [**Исследовательский анализ данных**](#2)
3. [**Портрет идеального кандидата**](#3)
4. [**Проверка гипотез**](#4)
5. [**Общий вывод**](#5)

Для выполнения проекта будет использованы следующие инструменты:

1. **Сбор данных** - библиотеки pyrogram и Beautiful Soup для парсинга сообщений из Telelgram и ссылок на вакансии
2. **Предобработка/исследовательский анализ** - библиотеки для анализа и визуализации (pandas/numpy/matplotlib/seaborn)
3. **Проверка гипотез** - статистический пакет функций scipy
4. **Презентация/Дашюорд** - программы Microsoft PowerPoint/Power BI

<a id='1'></a>
## Сбор и обзор данных

Для исследования были выбраны следующие карьерные каналы для специалистов Data Science:

- https://t.me/data_hr
- https://t.me/biheadhunter
- https://t.me/datajob
- https://t.me/analyst_job
- https://t.me/foranalysts
- https://t.me/bds_job
- https://t.me/datasciencejobs

Каналы содержат вакансии для различных специалистов в области науки о данных, таких как Data Scientist/Engineer, а также специальностей System/Business Analyst. В рамках исследования будут рассмотрены лишь вакансии аналитиков данных и смежных специалистов - Data/Product/BI Analyst. После парсинга данных сырые данные сразу будут отфильтрованы для подходящих вакансий. 

### Получение сырых данных

In [122]:
# !pip install pyrogram tgcrypto

In [123]:
# !pip install python-dotenv

In [124]:
# импортируем библиотеки для сбора и первичной обработки данных

import pyrogram
import requests
import nltk
import json
import re
import os

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from dotenv import load_dotenv
from pyrogram import Client
from tqdm.auto import tqdm 
tqdm.pandas()

In [125]:
# подготавливаем данные для парсера

dotenv_path = os.path.join('dot.env')

if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)

os.environ['DEMO']

API_ID = os.environ['API_ID']
API_HASH = os.environ['API_HASH']

In [126]:
# скрипт парсера

# from pyrogram import Client
# import os
# from dotenv import load_dotenv
# import pandas as pd


# path = os.path.dirname(os.path.abspath(__file__))

# dotenv_path = os.path.join(path + '/dot.env')
# if os.path.exists(dotenv_path):
#     load_dotenv(dotenv_path)
    
# API_ID = os.environ['API_ID']
# API_HASH = os.environ['API_HASH']

# targets = ['data_hr', 'biheadhunter', 'datajob', 'analyst_job', 
# 'foranalysts','bds_job', 'datasciencejobs']
# all_messages = []

# try:
#     with Client("my_account", API_ID, API_HASH) as app:
#         for target in targets:
#             for message in app.get_chat_history(target, limit=2000):
#                 all_messages.append([message.sender_chat, message.id, message.date, message.text, message.entities])
    
#     df = pd.DataFrame(all_messages)
#     df.columns = ["chat", "message_id", "date", "text", "entities"]
#     df.to_csv(path + '/data.csv', index=False)
#     print('Success: ', path + '/data.csv')
# except Exception as e:
#     print('Error: ', e)

Запускаем скрипт парсера из терминала:

In [127]:
# !python parser_script.py

In [128]:
# сохраняем данные и выводим основную информацию

data = pd.read_csv('parsed_data.csv', parse_dates=['date'])

data.info()
data.sample(5, random_state=0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6234 entries, 0 to 6233
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   chat        4239 non-null   object        
 1   message_id  6234 non-null   int64         
 2   date        6234 non-null   datetime64[ns]
 3   text        5604 non-null   object        
 4   entities    5489 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 243.6+ KB


,chat,message_id,date,text,entities
5828,"{\n ""_"": ""Chat"",\n ""id"": -1001321264581,...",454,2021-04-03 10:02:25,#office #moscow #москва #vacancy \n\nТребуется...,"[\n {\n ""_"": ""MessageEntity"",\n ..."
661,"{\n ""_"": ""Chat"",\n ""id"": -1001291755040,...",275,2020-10-25 14:23:06,@helen09977,"[\n {\n ""_"": ""MessageEntity"",\n ..."
1189,"{\n ""_"": ""Chat"",\n ""id"": -1001483488834,...",240,2021-10-24 19:00:05,👔 Data scientist\n⛳️ Форпост \n💼🥁 Работа...,"[\n {\n ""_"": ""MessageEntity"",\n ..."
2579,NaN,59438,2022-09-05 11:46:31,#вакансия #естьработа #удаленка #релокация #а...,"[\n {\n ""_"": ""MessageEntity"",\n ..."
2165,NaN,60162,2022-09-19 10:24:08,#вакансия #системный_аналитик #любой_город #...,"[\n {\n ""_"": ""MessageEntity"",\n ..."


Для оптимизации работы с текстом сразу избавимся от эмодзи:

In [129]:
# удалим данные с пропущенным значение в столбце 'text'

data.dropna(subset=['text'], inplace=True)

In [130]:
# напишем функцию для удаления эмодзи

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


data['text'] = data['text'].apply(remove_emoji)

In [147]:
test = 'DATA SCIENTIST'

ds = r'data'

if re.search(val, test.lower()):
             print('ну ок')
else:
    print('нихуя')

нихуя


'data scientist'

Получили сырые данные, состоящие из 6,234 строк и 5 столбцов. Для оптимизации дальнейшей обработки попробуем сразу отфильтровать вакансии для аналитиков данных с помощью регулярного выражения:

In [131]:
# пишем регулярное выражения для поиска подходящих вакансий

da = r'data analyst|data analysis|аналитик данных|анализ данных|#da|A/B'
pa = r'product analyst|продуктовый анализ|продуктовый аналитик'
bi = r'datalens|tableau|power bi|bi analyst'
ds = r'scien|дс|саен|data scientist|pytorch|машинное обучение|learning|vision|nlp'
de = r'engineer|инженер|spark|airflow|hadooop|scala|#de|etl|linux|архитект[оy]р[аы]|хранилище|разработчик|desktop'

# создаём столбцы для фильтрации данных

jobs = ['da', 'pa', 'bi', 'de', 'ds']

for val in jobs:
    
    data[val] = (
        data['text']
        .apply(
            lambda x: True if re.search(val, x.lower()) else False)
    )

# смотрим на результат

for vals in jobs:
    display(data[vals].value_counts())

False    3154
True     2450
Name: da, dtype: int64

False    4287
True     1317
Name: pa, dtype: int64

False    4123
True     1481
Name: bi, dtype: int64

False    4220
True     1384
Name: de, dtype: int64

False    5101
True      503
Name: ds, dtype: int64

Так как распределение между вариантами профессии неравномерно, сравнивать их между собой нецелесообразно - дальнейшее исследование проведём без разделения:

In [119]:
# удаляем лишние вакансии

data = (
    data
    .query('(ds == False)') #(da == True|pa == True|bi == True) & 
    .drop(columns=['da', 'pa', 'bi'])
)

# смотрим на результат

data.info()
data.sample(15, random_state=0)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5101 entries, 0 to 6232
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   chat        3351 non-null   object        
 1   message_id  5101 non-null   int64         
 2   date        5101 non-null   datetime64[ns]
 3   text        5101 non-null   object        
 4   entities    4988 non-null   object        
 5   de          5101 non-null   bool          
 6   ds          5101 non-null   bool          
dtypes: bool(2), datetime64[ns](1), int64(1), object(3)
memory usage: 249.1+ KB


,chat,message_id,date,text,entities,de,ds
3106,NaN,58616,2022-08-15 15:50:22,#outstaff #available #1C ...,"[\n {\n ""_"": ""MessageEntity"",\n ...",False,False
388,"{\n ""_"": ""Chat"",\n ""id"": -1001291755040,...",592,2022-01-21 18:08:34,#удаленно #BI \n#BusinessAnalyst\n#DataAnalys...,"[\n {\n ""_"": ""MessageEntity"",\n ...",False,False
5400,"{\n ""_"": ""Chat"",\n ""id"": -1001321264581,...",893,2022-03-20 09:30:50,#Вакансия #удаленка #DataEngineer #python #cli...,"[\n {\n ""_"": ""MessageEntity"",\n ...",True,False
1470,NaN,61335,2022-10-17 09:22:32,#вакансия #аналитик #blockchain #вилка #офис\n...,"[\n {\n ""_"": ""MessageEntity"",\n ...",False,False
2522,NaN,59515,2022-09-06 13:41:53,https://docs.google.com/presentation/d/e/2PACX...,"[\n {\n ""_"": ""MessageEntity"",\n ...",True,False
3432,"{\n ""_"": ""Chat"",\n ""id"": -1001137236002,...",4843,2022-10-11 11:05:09,Data Science consultant\nв Yandex.Cloud — пуб...,"[\n {\n ""_"": ""MessageEntity"",\n ...",True,False
2355,NaN,59830,2022-09-12 13:08:46,#вакансия #System #analyst #fulltime #Senior #...,"[\n {\n ""_"": ""MessageEntity"",\n ...",False,False
6172,"{\n ""_"": ""Chat"",\n ""id"": -1001321264581,...",71,2020-03-05 22:21:58,#вакансия #москва #mailru\n\n Две вакансии от ...,"[\n {\n ""_"": ""MessageEntity"",\n ...",True,False
864,"{\n ""_"": ""Chat"",\n ""id"": -1001291755040,...",50,2020-07-13 18:27:13,Предложение о работе на проекте банка ВТБ \n\n...,"[\n {\n ""_"": ""MessageEntity"",\n ...",True,False
2067,NaN,60303,2022-09-22 10:13:53,#вакансия #job #работа #system #analyst #syste...,"[\n {\n ""_"": ""MessageEntity"",\n ...",False,False


### Предобработка

Создадим несколько дополнительных столбцов - с названием чата, ссылками на полное описание вакансии, грейдом требуемого специалиста и наличием требований к образованию:

In [10]:
# заполним пропущенные значения в поле `chat`

data = (
    data
    .fillna(json.dumps({'title':'unspecified'}))
    .reset_index(drop=True)
)

# добавим столбец с названием чата

data["chat_title"] = (
    data["chat"]
    .apply(lambda x: json.loads(x)['title'])
)

In [11]:
# посмотрим на распределение вакансий по названиям чатов

data["chat_title"].value_counts()

Data Science Jobs / AI / NN / ML / DL / NLP                                                   710
unspecified                                                                                   700
Big Data Science job                                                                          522
Business Intelligence HeadHunter                                                              388
Data jobs — вакансии по data science, анализу данных, аналитике, искусственному интеллекту    357
Job for Analysts & Data Scientists                                                            352
Data Analytics Jobs                                                                           215
ALTSHIFT                                                                                        2
Name: chat_title, dtype: int64

In [12]:
# пишем функцию для извлечения ссылок из текста сообщений
# и добавляем в отдельный столбец

def get_links(x, regexp='http\S+'):
    try:
        for i in json.loads(x['entities']):
            if i['type'] == 'MessageEntityType.TEXT_LINK':
                return i['url']
            elif i['type'] == 'MessageEntityType.URL':
                url = re.findall(regexp, x['text'])[0]
                return url        
    except:
        return None
    
data['link'] = data.apply(get_links, axis=1)

In [13]:
# добавляем столбцы с наличием требования к образованию и опыту

educ = r'(технич|математ|прикладн|информат)'

data['education'] = data['text'].apply(lambda x: True if re.search(educ, x.lower()) else False)
data['no_exp'] = data['text'].apply(lambda x: True if re.search(r'без опыта', x.lower()) else False)

data['education'].value_counts()
data['no_exp'].value_counts()

False    3234
True       12
Name: no_exp, dtype: int64

In [34]:
# пишем функцию для получения грейда специалиста

def get_grade(text):
    
    junior = r'(junior|джун|начинающ|стаж[её]р|младший|без опыта|выпускник)'
    senior = r'(senior|синьор|сеньор|\Wстарш|Wlead\W|\Whead\W|\Wлид\W|\Wведущ)'
    
    if re.search(junior, text.lower()) != None:
        return 'junior'
    if re.search(senior, text.lower()) != None:
        return 'senior'
    else:
        return 'middle'

data['grade'] = data['text'].apply(get_grade)

data['grade'].value_counts()

middle    2098
senior     823
junior     325
Name: grade, dtype: int64

In [15]:
# пишем функцию для парсинга ссылок

def parse_site(url, element='div', extra_info=None):
    try:
        data = requests.get(url).text ###
        soup = BeautifulSoup(data)
        elements = []
        for extra in extra_info:
            elements += soup.find_all(element, extra)
        html = ''
        for i in elements:
            html += str(i)
        return html
    except Exception as e:
        return '<p>Не удалось получить элемент</p>'

In [17]:
# заранее добавили известные классы на рассматриваемых сайтах

# extra_infos = [{'class' : 'list-parent vacancy-desc'}, # proglib
#                {'class' : 'vacancy-description'}, ## hh.ru
#                {'class' : 'lc-jobs-vacancy__description'}, # yandex
#                {'class':'vacancy_vacancyInfo__27MOq'}, # aliftech
#                {'class':'lc-group__content lc-group__content_justify_start lc-group__content_viewport lc-screen lpc-section'}, # practicum.yandex
#                {'class' : 'wrapper1'}, # https://data.analyst.elama.team/
#                {'class' : 'container b-vacancy-v2'}, # getmatch.ru
#                {'class': 'job-page__text job-page__text_min-height'}, # profi.ru
#                {'itemprop' : 'description'}, # rabota.ru
#                {'class' : 'col-main entry-content'},# te-st.ru
#                {'class' : 'vacancy-section'}, # hh.ru        
#                {'class' : 'bloko-columns-row'}
#               ]

# # парсим ссылки

# data['parsed_text'] = (
#     data['link']
#     .progress_apply(lambda x: parse_site(x, extra_info=extra_infos))
# )

In [18]:
# финальный вариант

data.info()
data.sample(5, random_state=0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3246 entries, 0 to 3245
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   chat        3246 non-null   object        
 1   message_id  3246 non-null   int64         
 2   date        3246 non-null   datetime64[ns]
 3   text        3246 non-null   object        
 4   entities    3246 non-null   object        
 5   chat_title  3246 non-null   object        
 6   link        1611 non-null   object        
 7   education   3246 non-null   bool          
 8   no_exp      3246 non-null   bool          
 9   grade       3246 non-null   object        
dtypes: bool(2), datetime64[ns](1), int64(1), object(6)
memory usage: 209.3+ KB


,chat,message_id,date,text,entities,chat_title,link,education,no_exp,grade
1559,"{""title"": ""unspecified""}",58668,2022-08-16 09:23:09,#вакансия #lead\nВакансия: Lead System Analyst...,"[\n {\n ""_"": ""MessageEntity"",\n ...",unspecified,None,False,False,senior
716,"{\n ""_"": ""Chat"",\n ""id"": -1001483488834,...",334,2022-03-13 19:00:04,👔 Junior Data Engineer\n⛳️ Метр Квадратный \n...,"[\n {\n ""_"": ""MessageEntity"",\n ...","Data jobs — вакансии по data science, анализу ...",https://proglib.io/vacancies/junior-data-engin...,False,False,junior
1726,"{\n ""_"": ""Chat"",\n ""id"": -1001137236002,...",4681,2022-08-23 15:12:10,Data Analyst\nв Dodo Brands — компания объедин...,"[\n {\n ""_"": ""MessageEntity"",\n ...",Job for Analysts & Data Scientists,https://geekjob.ru/hYx9,False,False,tbd
2621,"{\n ""_"": ""Chat"",\n ""id"": -1001321264581,...",975,2022-05-28 09:20:41,#вакансия #удалённо #job #remote #parttime #пр...,"[\n {\n ""_"": ""MessageEntity"",\n ...",Data Science Jobs / AI / NN / ML / DL / NLP,https://otus.ru/lessons/data-engineer/,False,False,junior
298,"{\n ""_"": ""Chat"",\n ""id"": -1001291755040,...",553,2021-11-01 12:54:29,SmartDataLab\n\nищет разработчиков MS SQL на п...,"[\n {\n ""_"": ""MessageEntity"",\n ...",Business Intelligence HeadHunter,None,True,False,tbd


-

<a id='2'></a>
## Исследовательский анализ данных

<a id='3'></a>
## Портрет идеального кандидата

<a id='4'></a>
## Проверка гипотез

<a id='6'></a>
## Общий вывод